In [ ]:
import os
import math
import numpy as np
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
import pickle
from tqdm import tqdm
import matplotlib.pyplot as plt

## Preludium for transforming meteo

In [ ]:
Wh_to_kJ = lambda x: x * 86.4
tdew_to_kpa = lambda x: ea_from_tdew(x)
hr_n_t_to_hpa = lambda hr, t: vapor_pressure(t, hr)
to_date = lambda d: d.date()


def vapor_pressure(T, RH):
    """
    Compute actual vapor pressure from temperature and relative humidity.
    
    Parameters:
        T : float or np.array
            Temperature in Celsius
        RH : float or np.array
            Relative Humidity in %
    
    Returns:
        e : float or np.array
            Actual vapor pressure in hPa
    """
    # Saturation vapor pressure (hPa)
    es = 6.112 * np.exp((17.67 * T) / (T + 243.5))
    
    # Actual vapor pressure (hPa)
    e = RH / 100 * es
    return e

def ea_from_tdew(tdew):
    """
    Calculates actual vapour pressure, ea [kPa] from the dewpoint temperature
    using equation (14) in the FAO paper. As the dewpoint temperature is the
    temperature to which air needs to be cooled to make it saturated, the
    actual vapour pressure is the saturation vapour pressure at the dewpoint
    temperature. This method is preferable to calculating vapour pressure from
    minimum temperature.

    Taken from fao_et0.py written by Mark Richards

    Reference:
    Allen, R.G., Pereira, L.S., Raes, D. and Smith, M. (1998) Crop
        evapotranspiration. Guidelines for computing crop water requirements,
        FAO irrigation and drainage paper 56)

    Arguments:
    tdew - dewpoint temperature [deg C]

    Post-Comment:
    I take this function from the psce python module.
    """
    # Raise exception:
    if tdew < -95.0 or tdew > 65.0:
        # Are these reasonable bounds?
        msg = 'tdew=%g is not in range -95 to +60 deg C' % tdew
        raise ValueError(msg)

    tmp = (17.27 * tdew) / (tdew + 237.3)
    ea = 0.6108 * math.exp(tmp)
    return ea


def plots_coords_to_WOFcsv(df_plot):
    # In each line the comment indicates the unit
    df_pcse = pd.DataFrame({"DAY": df_plot["date_mesure"].dt.strftime("%Y%m%d"), # YYYYMMDD
                            "TMAX": df_plot["T2M_MAX"], # °C
                            "TMIN": df_plot["T2M_MIN"], # °C
                            "TEMP": df_plot["T2M_MEAN"],# °C
                            "IRRAD": (df_plot["SSI_MEAN"].apply(Wh_to_kJ)).round(2), #kJ/m^2
                            "RAIN": df_plot["PRECIP_SUM"], # mm
                            "WIND": df_plot["WS2M_MEAN"], 
                            "VAP": (df_plot["DEWT2M_MEAN"].apply(tdew_to_kpa)).round(2), # hpa
                            "SNOWDEPTH": "NaN", # cm
                            })
    # Consistency test for lon, lat & id 
    lon = (df_plot["Longitude"].unique())[0] if len(df_plot["Longitude"].unique()) == 1 else print("Consistency error in the coords DF - lon")
    lat = (df_plot["Latitude"].unique())[0] if len(df_plot["Latitude"].unique()) == 1 else print("Consistency error in the coords DF - lat")
    id_plot = (df_plot["PlotId"].unique())[0] if len(df_plot["PlotId"].unique()) == 1 else print("Consistency error in the coords DF - id")
    plot_info = {"ID" : id_plot,
                 "LON": lon,
                 "LAT": lat,
                 "ELEV": 30}
    
    return df_pcse, plot_info

# For one scenario

**Disclaimer**: Following lines don't work because they refer to data that we cannot share for reasons of personal data protection. Instead, we propose an alternative solution from [Here](#you-can-normally-executed-the-code-from-here)

## Import data

In [ ]:
with open("../pg-sequence_learning_work_v/Agrial_DL_df/data/sims_setup.pickle", "rb") as f:
    sims_setup = pickle.load(f)

In [ ]:
sims_setup.loc[sims_setup["real_crop"] == "Blé tendre d'hiver"]
# 011a0bab-5f91-4784-a7b5-eb895dcdbcda
selected_latitude = sims_setup.loc[sims_setup["id"] == "011a0bab-5f91-4784-a7b5-eb895dcdbcda","latitude"].values[0]
selected_longitude = sims_setup.loc[sims_setup["id"] == "011a0bab-5f91-4784-a7b5-eb895dcdbcda","longitude"].values[0]

objective_plot = Point(selected_longitude, selected_latitude)

In [ ]:
grid = gpd.read_file("data/drias2020/drias2020.shp")

In [ ]:
objective_plot.x, objective_plot.y

## For reasons of personal data protection, we extract non-sensitive information from this plot so that the experiment can be reproduced.

In [ ]:
sims_setup = sims_setup.loc[sims_setup["id"] == "011a0bab-5f91-4784-a7b5-eb895dcdbcda", ["id", "latitude", "longitude", "crop", "variety"]]

with open("data/selected_plot.pkl", "wb") as f:
    pickle.dump(sims_setup, f)

## You can normally executed the code from here

In [ ]:
with open("data/selected_plot.pkl", "rb") as f:
    sims_setup = pickle.load(f)

In [ ]:
sims_setup.loc[sims_setup["id"] == "011a0bab-5f91-4784-a7b5-eb895dcdbcda", "crop_start_date"] = pd.to_datetime("2035-10-01")
sims_setup.loc[sims_setup["id"] == "011a0bab-5f91-4784-a7b5-eb895dcdbcda", "crop_end_date"] = pd.to_datetime("2036-08-01")

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10))
grid.plot(ax=ax)
plt.scatter(objective_plot.x, objective_plot.y, color="red", s=10)


In [ ]:
containing_polygon = grid[grid.contains(objective_plot)]

In [ ]:
containing_polygon.columns

In [ ]:
meteo_26 = pd.read_csv("data/siclima_extraction_3061.csv", sep=";")
meteo_45 = pd.read_csv("data/siclima_extraction_3062.csv", sep=";")
meteo_85 = pd.read_csv("data/siclima_extraction_3063.csv", sep=";")

In [ ]:
dico = {"26": meteo_26,
                          "45": meteo_45,
                          "85": meteo_85}

In [ ]:
list(dico.items())[0][0]

In [ ]:
for meteo_name, meteo in list({"26": meteo_26,
                               "45": meteo_45,
                               "85": meteo_85}.items()):
    meteo = meteo[meteo["cell"] == 14910].reset_index(drop=True)
    meteo["day"] = meteo["day_of_month"]
    meteo["date"] = pd.to_datetime(meteo[['year', 'month', 'day']])
    meteo["TMAX"] = meteo["tasmax"] # C
    meteo["TMIN"] = meteo["tasmin"] # C
    meteo["TEMP"] = meteo["tas"] # C
    meteo["IRRAD"] = (meteo["rlds"].apply(Wh_to_kJ)).round(2) # W/m^2
    meteo["RAIN"] = meteo["prtot"] 
    meteo["WIND"] = meteo["sfcwind"] # m/s
    meteo["VAP"] = hr_n_t_to_hpa(meteo["hr"], meteo["tas"]) # hPa

    meteo["id"] = "011a0bab-5f91-4784-a7b5-eb895dcdbcda"

    ts_data = meteo.loc[:, ["id", "date", "TMAX", "TMIN", "TEMP", "IRRAD", "RAIN", "WIND", "VAP"]]
    ts_data.set_index("id", inplace=True)

    DAYS_BEFORE_SOWING=30

    grouped_ts = ts_data.groupby("id")
    sowing_dates = sims_setup.loc[sims_setup["id"] == "011a0bab-5f91-4784-a7b5-eb895dcdbcda"].set_index("id")["crop_start_date"]
    unique_ids = sowing_dates.index.values

    before_ts = np.zeros((len(unique_ids), DAYS_BEFORE_SOWING, ts_data.shape[1] - 1))
    before_mask = np.zeros((len(unique_ids), DAYS_BEFORE_SOWING, ts_data.shape[1] - 1), dtype=np.int8)
    after_ts = np.zeros((len(unique_ids), 200, ts_data.shape[1] - 1))
    after_mask = np.zeros((len(unique_ids), 200, ts_data.shape[1] - 1), dtype=np.int8)

    for idx, id in enumerate(tqdm(unique_ids)):
        if id not in grouped_ts.groups:
            print(f"Missing ID {id} in ts_data.")

        sowing_date = sowing_dates.loc[id]
        df = grouped_ts.get_group(id)
        df_values = df.drop(columns=["date"]).values
        df_dates = df["date"].values

        mask_before = (df_dates >= sowing_date - pd.Timedelta(days=DAYS_BEFORE_SOWING)) & \
                    (df_dates < sowing_date)
        mask_after = (df_dates >= sowing_date) & \
                    (df_dates < sowing_date + pd.Timedelta(days=200))

        before_vals = df_values[mask_before]
        after_vals = df_values[mask_after]

        if before_vals.shape[0] == DAYS_BEFORE_SOWING:
            before_ts[idx] = before_vals
            before_mask[idx] = 1
        else:
            print(f"Warning: ID {id} has {before_vals.shape[0]} days before sowing, expected {DAYS_BEFORE_SOWING}. APLYING PADDING.")
            before_ts[idx, :before_vals.shape[0]] = before_vals

        if after_vals.shape[0] == 200:
            after_ts[idx] = after_vals
            after_mask[idx] = 1
        else:
            print(f"Warning: ID {id} has {after_vals.shape[0]} days after sowing, expected 200. APLYING PADDING.")
            after_ts[idx, :after_vals.shape[0]] = after_vals
    
    print("before ts : ", before_ts.shape)
    print("before mask : ", before_mask.shape)
    print("after ts : ", after_ts.shape)
    print("after mask : ", after_mask.shape)

    for array in ["before_ts", "before_mask", "after_ts", "after_mask"]:
        os.makedirs(name=f"data/work_data/RCP_{meteo_name}/", exist_ok=True)
        with open(f"data/work_data/RCP_{meteo_name}/{array}.pkl", "wb") as f:
            pickle.dump(eval(array), f)